### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.rcParams['figure.figsize'] = [16,6]

### Importing Datset

In [3]:
ipl_df = pd.read_csv('all_matches.csv')
ipl_df.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
ipl_df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

### Feature Engineering

In [5]:
ipl_df = ipl_df[(ipl_df.innings == 1) | (ipl_df.innings == 2)]

In [6]:
ipl_df['total_runs'] = ipl_df.apply(lambda x: x['runs_off_bat'] + x['extras'], axis= 1)

In [7]:
ipl_df['isOut'] = ipl_df['player_dismissed'].apply(lambda x: 1 if type(x) == type('str') else 0)

In [8]:
ipl_df.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,total_runs,isOut
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,0
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [9]:
ipl_df['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Deccan Chargers', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [10]:
t1 = 'Gujarat Titans'
t2 = 'Chennai Super Kings'

In [12]:
ipl_df[(ipl_df['batting_team'] == t1) & (ipl_df['bowling_team'] == t2) & (ipl_df['season'] == 2023)].match_id.unique()

array([1359475, 1370350, 1370353], dtype=int64)

In [13]:
matchID = 1370353

In [14]:
outcomes = [0,1,2,3,4,6,'w']

In [16]:
t1_outcomes_count = ipl_df[ipl_df.batting_team == t1].total_runs.value_counts()
t2_outcomes_count = ipl_df[ipl_df.batting_team == t2].total_runs.value_counts()

In [17]:
t1_outs = ipl_df[ipl_df.batting_team == t1].isOut.sum()
t2_outs = ipl_df[ipl_df.batting_team == t2].isOut.sum()

In [18]:
t1_outcomes = []
t2_outcomes = []

for outcome in outcomes:
    if outcome != 'w':
        t1_outcomes.append(t1_outcomes_count[outcome])
        t2_outcomes.append(t2_outcomes_count[outcome])
    else:
        t1_outcomes.append(t1_outs)
        t2_outcomes.append(t2_outs)

In [19]:
t1_outcomes, t2_outcomes

([1662, 2447, 387, 14, 698, 270, 247],
 [9676, 12105, 1955, 111, 3240, 1499, 1243])

In [20]:
t1_pb_outcomes = [round(i/sum(t1_outcomes), 5) for i in t1_outcomes]
t2_pb_outcomes = [round(i/sum(t2_outcomes), 5) for i in t2_outcomes]

In [21]:
t1_pb_outcomes

[0.29031, 0.42742, 0.0676, 0.00245, 0.12192, 0.04716, 0.04314]

In [22]:
t2_pb_outcomes

[0.32438, 0.40581, 0.06554, 0.00372, 0.10862, 0.05025, 0.04167]

### Fetching Probability Values

In [24]:
def get_pbvalues(teamName):
    if teamName == 'GT':
        p_0 = 0.29031
        p_1 = 0.29031 + 0.42742
        p_2 = 0.29031 + 0.42742 + 0.0676
        p_3 = 0.29031 + 0.42742 + 0.0676 + 0.00245
        p_4 = 0.29031 + 0.42742 + 0.0676 + 0.00245 + 0.12192
        p_6 = 0.29031 + 0.42742 + 0.0676 + 0.00245 + 0.12192 + 0.04716
        p_w = 1
        
    elif teamName == 'CSK':
        p_0 = 0.32438
        p_1 = 0.32438 + 0.40581
        p_2 = 0.32438 + 0.40581 + 0.06554
        p_3 = 0.32438 + 0.40581 + 0.06554 + 0.00372
        p_4 = 0.32438 + 0.40581 + 0.06554 + 0.00372 + 0.10862
        p_6 = 0.32438 + 0.40581 + 0.06554 + 0.00372 + 0.10862 + 0.05025
        p_w = 1
        
    return p_0, p_1, p_2, p_3, p_4, p_6, p_w

###  Runs Prediction Model

In [25]:
def predict_runs(target, current_score, current_wickets, current_overs, max_over):
    i1p_0, i1p_1, i1p_2, i1p_3, i1p_4, i1p_6, i1p_w = get_pbvalues('GT')
    i2p_0, i2p_1, i2p_2, i2p_3, i2p_4, i2p_6, i2p_w = get_pbvalues('CSK')
    
    pred_runs = current_score
    pred_wks = current_wickets
    leftover_balls = (max_over*6) - current_overs*6
    
    for i in range(leftover_balls):
        r_value = np.random.random()
        
        if r_value < i2p_0:
            pred_runs += 0
        elif r_value < i2p_1:
            pred_runs += 1
        elif r_value < i2p_2:
            pred_runs += 2
        elif r_value < i2p_3:
            pred_runs += 3
        elif r_value < i2p_4:
            pred_runs += 4
        elif r_value < i2p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            
            if pred_wks == 10:
                break
            
        if pred_runs > target:
            break
            
    return pred_runs

In [26]:
predict_runs(170, 72, 0, 6, 15)

151

### Winner Function

In [27]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [28]:
target = 170
current_score = 112
current_wickets = 2
current_overs = 10
max_over = 15

iter_count = 100

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0

for i in range(iter_count):
    pred_runs = predict_runs(target, current_score, current_wickets, current_overs, max_over)
    runs_ls.append(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)
    
    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count += 1

In [29]:
win_count, tie_count, lose_count

(2, 1, 97)

### Finding out Runs at Required Stages

In [30]:
def find_runs(current_score, target,current_wickets, at_overs, max_over):
    runs_ls = []
    result_ls = []
    
    req_runs = []
    win_ls = []
    
    for i in range(current_score, target + 1):
        win_count = 0
        tie_count = 0
        lose_count = 0
        
        for j in range(100):
            pred_runs = predict_runs(target, i, current_wickets, at_overs, max_over)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            result_ls.append(result_pred)
            
            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count += 1
            
            win_ls.append(win_count)
            req_runs.append(i)
            
    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            break
            
    return required_runs

In [31]:
find_runs(112, 170, 2, 10, 15)

134

### Finding out Wickets a Team can afford to Lose

In [32]:
def find_wickets(current_score, target, current_wickets, at_overs, max_over):
    req_runs = find_runs(current_score, target, current_wickets, at_overs, max_over)
    
    runs_ls = []
    results_ls = []
    
    win_ls = []
    req_wkts = []
    
    for i in range(current_wickets, 10):
        win_count = 0
        tie_count = 0
        lose_count = 0
        
        for j in range(100):
            pred_runs = predict_runs(target, current_score, i, at_overs, max_over)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)
            
            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count += 1
                
        win_ls.append(win_count)
        req_wkts.append(i)
        
    required_wicket_value = current_wickets
    
    for i in range(len(req_wkts)):
        if (win_ls[i] < 50):
            required_wicket_value = req_wkts[i]
            break
            
    return required_wicket_value

In [33]:
find_wickets(150, 170, 1, 10, 15)

1

### Function: Interactive Chart

In [42]:
def find_runs_wickets(current_wks, at_overs, target_score, max_over):
    x = np.array(list(range(21)))
    
    req_value = find_runs(112, target_score, current_wks, at_overs, max_over)
    req_wk_value = find_wickets(112, target_score, current_wks, at_overs, max_over)
    
    if at_overs == 10:
        req_value = 112
        req_wk_value = 2
        
    y = np.array([req_value for i in range(21)])
    
    plt.scatter(at_overs, req_value, s= 1200, color= 'gold')
    plt.axhline(target_score, ls= '--', color= 'blue')
    plt.text(1, target_score + 10, 'Target Score :' + str(target_score), color= 'darkblue', fontsize= 13)
    plt.text(at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color= 'blue', fontsize= 13, horizontalalignment= 'center', verticalalignment= 'center')
    plt.text(at_overs, req_value-30, 'CSK has to be at ' + str(req_value) + '/' + str(req_wk_value) + 'after' + str(at_overs) + 'ov', horizontalalignment= 'center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be: ', fontsize= 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

### Interactive Visualization

In [43]:
print('Current Score= CSK: 112/2 (10 overs)')
print('')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min= 1, max= 10, step= 1, value= 1), at_overs= widgets.IntSlider(min= 1, max= max_over-1, step= 1, value= 10), target_score= widgets.IntSlider(min= 1, max= 300, step= 1, value= 165), max_over= widgets.IntSlider(min= 1, max= 20, step= 1, value= 10))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

Current Score= CSK: 112/2 (10 overs)



interactive(children=(IntSlider(value=1, description='current_wks', max=10, min=1), IntSlider(value=10, descri…